In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alpha_vantage
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.models import Sequential
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Lasso
from pandas import Series
import time
import csv 
import json
import sys
from sklearn.model_selection import TimeSeriesSplit
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score,mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from ta import *
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn import linear_model
from sklearn.pipeline import Pipeline
from IPython.display import display
import io
import os
from keras.utils import plot_model,model_to_dot
from IPython.display import SVG
from datetime import datetime
import tensorflow as tf
from tensorflow import keras





data = pd.DataFrame()
data = pd.read_csv('3573OrangeJuiceDemand.csv', header=0, encoding = 'unicode_escape', index_col='Row Labels')
data = data.drop(['Rounded # of Boxes', 'day of week','Unnamed: 8','Unnamed: 9'], axis=1)
data.dropna()
print(data)




def train_test_split_lstm(days, prediction_time=1, test_data_size=133, unroll_length=6):
    
    # training data
    test_data_cut = test_data_size + unroll_length + 1

    x_train = days[0:-prediction_time - test_data_cut].as_matrix()
    y_train = days[prediction_time:-test_data_cut]['# of Boxes'].as_matrix()

    # test data
    x_test = days[0 - test_data_cut:-prediction_time].as_matrix()
    y_test = days[prediction_time - test_data_cut:]['# of Boxes'].values()

    return x_train, x_test, y_train, y_test

# sequence_length=12<-do not erase this 
def unroll(data, sequence_length=6):
    
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])
    return np.asarray(result)
#print(data1_normalized.head(1))
xTrain, xTest,yTrain, yTest = train_test_split_lstm(data)

unroll_length = 6

xTrain = unroll(xTrain, unroll_length)
xTest = unroll(xTest, unroll_length)
yTrain = yTrain[-xTrain.shape[0]:]
yTest = yTest[-xTest.shape[0]:]


## Get the shapes of the data
print("x_train", xTrain.shape)
print("y_train", yTrain.shape)
print("x_test", xTest.shape)
print("y_test", yTest.shape)
os.environ['KMP_DUPLICATE_LIB_OK']='True'


            OJ  Grand Total  numerical  weekend  # of Boxes
Row Labels                                                 
10/7/2015   56           56          3        0    2.333333
10/8/2015   50           50          4        0    2.083333
10/9/2015   52           52          5        1    2.166667
10/10/2015  85           85          6        1    3.541667
10/12/2015  45           45          1        0    1.875000
...         ..          ...        ...      ...         ...
1/22/2020   45           45          3        0    1.875000
1/23/2020   47           47          4        0    1.958333
1/24/2020   63           63          5        1    2.625000
1/25/2020   89           89          6        1    3.708333
1/27/2020   41           41          1        0    1.708333

[1327 rows x 5 columns]


AttributeError: 'DataFrame' object has no attribute 'as_matrix'

In [ ]:
def build_basic_model(input_dim, output_dim, return_sequences):
    
    model = Sequential()
    model.add(LSTM(
        input_shape=(None, input_dim),
        units=output_dim,
        return_sequences=return_sequences))

    model.add(LSTM(
        128,
        return_sequences=False))

    model.add(Dense(
        units=1))
    model.add(Activation('linear'))
#     comment below out I am just seeing what it does
#    model.add(Dropout(0.4))
    
  #  model.add(Dense(
 #         units=1))
#    model.add(Activation('linear'))
    
#    model.add(Dense(
#        units=1))
#    model.add(Activation('relu'))

    return model

model1=build_basic_model(input_dim = xTrain.shape[-1],output_dim = unroll_length, return_sequences=True)

# Compile the model
start = time.time()
# model.compile(loss='mean_squared_error', optimizer='adagrad')
# Note:adding 'adagrad' optimizer and dropout resulted in : -> 
# continued..:Forget about the dropout and: the optimizer is not good but for complex models from what it looks like

# Below is the original
# model.compile(loss='mean_squared_error', optimizer='nadam',metrics=['accuracy'])
model1.compile(loss='mean_squared_error', optimizer='nadam')
print('compilation time : ', time.time() - start)

model1.fit(xTrain,yTrain,epochs=100,validation_split=.03)

predictions = model1.predict(xTest)

predlist = []
for item in predictions.tolist():
    predlist.append(item[0])
testlist = yTest.tolist()



df = pd.DataFrame(list(zip(predlist,testlist)),columns = ['prediction','actual'])
df.to_csv('3573OJPredictionvsActual.csv')


        
    






def plot_lstm_prediction(actual, prediction, title='Orange Juice Demand Prediction', y_label='# of Boxes', x_label='Days'):
    
    fig = plt.figure()
    ax = fig.add_subplot(111)

    # Add labels
    plt.ylabel(y_label)
    plt.xlabel(x_label)

    # Plot actual and predicted close values

    plt.plot(actual, '#00FF00', label='Actual Demand')
    plt.plot(prediction, '#0000FF', label='Predicted Demand')

    # Set title
    ax.set_title(title)
    ax.legend(loc='lower left')


    plt.show()
plot_lstm_prediction(yTest,predictions)

trainScore = model1.evaluate(xTrain, yTrain, verbose=0)
print('Train Score: %.8f MSE (%.8f RMSE)' % (trainScore, math.sqrt(trainScore)))

testScore = model1.evaluate(xTest, yTest, verbose=0)
print('Test Score: %.8f MSE (%.8f RMSE)' % (testScore, math.sqrt(testScore)))

NameError: name 'xTrain' is not defined